[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset

Let's first of all have a look at the data. You can download the file `labeledTrainData.tsv` on the [Kaggle website of the competition](https://www.kaggle.com/c/word2vec-nlp-tutorial/data), or on our [Google Drive](https://drive.google.com/file/d/1a1Lyn7ihikk3klAX26fgO3YsGdWHWoK5/view?usp=sharing)


In [2]:
# Import pandas, numpy
import numpy as np
import pandas as pd

In [3]:
# Read dataset with extra params sep='\t', encoding="latin-1"
data = pd.read_csv('labeledTrainData.tsv', sep = '\t', encoding = "Latin-1")
data.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


## 2. Preprocessing

In [4]:
# stop words
data['id'].describe()


count       25000
unique      25000
top       7096_10
freq            1
Name: id, dtype: object

In [5]:
data.groupby('sentiment').describe().transpose()


sentiment                                                      0  \
id     count                                               12500   
       unique                                              12500   
       top                                                4088_1   
       freq                                                    1   
review count                                               12500   
       unique                                              12432   
       top     When i got this movie free from my job, along ...   
       freq                                                    3   

sentiment                                                      1  
id     count                                               12500  
       unique                                              12500  
       top                                               7096_10  
       freq                                                    1  
review count                                               12500  
       unique                                              12472  
       top     A have a female friend who is currently being ...  
       freq                                                    2

In [6]:
data['sentiment'].describe()

count    25000.00000
mean         0.50000
std          0.50001
min          0.00000
25%          0.00000
50%          0.50000
75%          1.00000
max          1.00000
Name: sentiment, dtype: float64

In [7]:
data['length'] = data['review'].apply(len)
data.head()

,id,sentiment,review,length
0,5814_8,1,With all this stuff going down at the moment w...,2302
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...",946
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,2455
3,3630_4,0,It must be assumed that those who praised this...,2245
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,2233


In [8]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    text = re.sub('<[^>]*>', '',text)
    # text = re.sub('@[a-z]')
    # Save emoticons for later appending
    # Your code here
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    # Your code here
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    return text

In [9]:
#stop words 
from collections import Counter

vocab = Counter()
for twit in data.review:
    for word in twit.split(' '):
        vocab[word] += 1

vocab.most_common(20)

[('the', 287025),
 ('a', 155092),
 ('and', 152651),
 ('of', 142970),
 ('to', 132568),
 ('is', 103227),
 ('in', 85576),
 ('I', 65971),
 ('that', 64555),
 ('this', 57195),
 ('it', 54416),
 ('/><br', 50935),
 ('was', 46697),
 ('as', 42509),
 ('with', 41718),
 ('for', 41067),
 ('but', 33783),
 ('The', 33760),
 ('on', 30765),
 ('movie', 30496)]

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\minha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

vocab_reduced = Counter()
# Go through all of the items of vocab using vocab.items() and pick only words that are not in 'stop_words' 
# and save them in vocab_reduced
for w, c in vocab.items():
    if not w in stop_words:
        vocab_reduced[w]=c

vocab_reduced.most_common(20)

[('I', 65971),
 ('/><br', 50935),
 ('The', 33760),
 ('movie', 30496),
 ('film', 27394),
 ('one', 20685),
 ('like', 18133),
 ('This', 12279),
 ('would', 11922),
 ('good', 11435),
 ('It', 10950),
 ('really', 10814),
 ('even', 10605),
 ('see', 10154),
 ('-', 9355),
 ('get', 8776),
 ('story', 8523),
 ('much', 8506),
 ('time', 7762),
 ('make', 7485)]

In [12]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
# Your code here
porter = PorterStemmer()

def tokenizer(text):
    return text.split()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()] # Your code here


In [13]:
# split the dataset in train and test

# Your code here
from sklearn.model_selection import train_test_split
X = data['review']
y = data['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=102)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [14]:
# Import Pipeline, LogisticRegression, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer


## 4. Evaluate Model

In [15]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
tfidf = TfidfVectorizer(stop_words=stop_words,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)

# A pipeline is what chains several steps together, once the initial exploration is done. 
# For example, some codes are meant to transform features — normalise numericals, or turn text into vectors, 
# or fill up missing data, they are transformers; other codes are meant to predict variables by fitting an algorithm,
# they are estimators. Pipeline chains all these together which can then be applied to training data
clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)


D:\ana\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Pipeline(steps=[('vect',
                 TfidfVectorizer(preprocessor=<function preprocessor at 0x00000192F61B6A60>,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...],
                                 tokenizer=<function tokenizer_porter at 0x00000192F5EFF550>)),
                ('clf', LogisticRegression(random_state=0))])

## 5. Export Model 

In [24]:
# Using pickle to export our trained model
import pickle
import os
pickle.dump(clf, open('output.pkl', 'wb'))

In [20]:
twits = [
    "I do not feel not bad", # Phuc +1
    'This model is "so good" :))', # Long -1
    'we are who we are', # Nghi 0
    'its good to be bad sometimes', # PA +1
    'what a wonderful failure! (sarcasm :)))', #Phuc +1
    'People do not like the bad things', # Chi 0
    'We finally have the test result. You are positive', # Long +1
]

preds = clf.predict_proba(twits)

for i in range(len(twits)):
    print(f'{twits[i]} --> Negative, Positive = {preds[i]}')

I do not feel not bad --> Negative, Positive = [0.97547349 0.02452651]
This model is "so good" :)) --> Negative, Positive = [0.2534306 0.7465694]
we are who we are --> Negative, Positive = [0.48591776 0.51408224]
its good to be bad sometimes --> Negative, Positive = [0.76957446 0.23042554]
what a wonderful failure! (sarcasm :))) --> Negative, Positive = [0.30298275 0.69701725]
People do not like the bad things --> Negative, Positive = [0.97163379 0.02836621]
We finally have the test result. You are positive --> Negative, Positive = [0.69362301 0.30637699]
